In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import librosa
from librosa import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time
import os
import joblib


In [ ]:

path = '/content/drive/My Drive/Colab_Notebooks/dataset/'
lst = []

start_time = time.time()

for subdir, dirs, files in os.walk(path):
  for file in files:
      # print(file)
      try:
        #Load librosa array, get mfcc
        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        #  converts the file name to gender
        file = int(file[18:20])
        print(file)
        if file%2==0:
          file=int(0)
          print("female")
        else: 
          file=int(1)
          print("male")
        arr = mfccs, file
        lst.append(arr)

      except ValueError:  # handling errors
        continue

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

14
female
14
female
14
female
14
female
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
15
male
16
female
16
female
15
male
15
male
15
male
15
male
16
female
16
female
16
female
16
female
16
female
16
female
16
female
16
female
16
female
16
female
16
femal

In [ ]:
len(lst)


2452

In [ ]:
X, Y = zip(*lst)  # unzipping the lst to the features and index of gender


In [ ]:
np.unique(Y, return_counts=True)

(array([0, 1]), array([1204, 1248]))

In [ ]:
# convert input into array
import numpy as np
X = np.asarray(X)
Y = np.asarray(Y)
print(X.shape, Y.shape)

(2452, 40) (2452,)


In [ ]:
# saving the Variables to drive
import joblib
X2 = 'X1.joblib'
Y2 = 'Y1.joblib'
save_dir = '/content/drive/My Drive/Colab_Notebooks/genderdata/XandY'
savedX = joblib.dump(X, os.path.join(save_dir, X2))
savedY = joblib.dump(Y, os.path.join(save_dir, Y2))

In [ ]:
# loading variables
import joblib
X = joblib.load('/content/drive/My Drive/Colab_Notebooks/genderdata/XandY/X1.joblib')
Y = joblib.load('/content/drive/My Drive/Colab_Notebooks/genderdata/XandY/Y1.joblib')

In [ ]:
np.unique(Y, return_counts=True)

(array([0, 1]), array([1204, 1248]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

In [ ]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)
print(x_traincnn.shape, x_testcnn.shape)

(1642, 40, 1) (810, 40, 1)


In [ ]:
# designing model
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(40,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(64, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(2)))

model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=1e-07, decay=0.0)

In [ ]:
model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 40, 256)           1536      
_________________________________________________________________
activation_108 (Activation)  (None, 40, 256)           0         
_________________________________________________________________
dropout_85 (Dropout)         (None, 40, 256)           0         
_________________________________________________________________
max_pooling1d_74 (MaxPooling (None, 10, 256)           0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 10, 128)           163968    
_________________________________________________________________
activation_109 (Activation)  (None, 10, 128)           0         
_________________________________________________________________
dropout_86 (Dropout)         (None, 10, 128)         

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
fitcnn=model.fit(x_traincnn, Y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, Y_test))

Epoch 1/500
83/83 [==============================] - 3s 29ms/step - loss: 1.5608 - accuracy: 0.6218 - val_loss: 0.6268 - val_accuracy: 0.7630
Epoch 2/500
83/83 [==============================] - 2s 26ms/step - loss: 0.8272 - accuracy: 0.7334 - val_loss: 0.6931 - val_accuracy: 0.7741
Epoch 3/500
83/83 [==============================] - 2s 25ms/step - loss: 0.6663 - accuracy: 0.7933 - val_loss: 0.2833 - val_accuracy: 0.8877
Epoch 4/500
83/83 [==============================] - 2s 26ms/step - loss: 0.4675 - accuracy: 0.8287 - val_loss: 0.2167 - val_accuracy: 0.9136
Epoch 5/500
83/83 [==============================] - 2s 26ms/step - loss: 0.4434 - accuracy: 0.8385 - val_loss: 0.2274 - val_accuracy: 0.9111
Epoch 6/500
83/83 [==============================] - 2s 25ms/step - loss: 0.3951 - accuracy: 0.8493 - val_loss: 0.2081 - val_accuracy: 0.9160
Epoch 7/500
83/83 [==============================] - 2s 26ms/step - loss: 0.3677 - accuracy: 0.8634 - val_loss: 0.2071 - val_accuracy: 0.9074
Epoch 

In [ ]:
# prediction
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
p = model.predict_classes(x_testcnn)
print(classification_report(Y_test,p))
print(confusion_matrix(Y_test,p)) # confusion matrix

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       372
           1       1.00      1.00      1.00       438

    accuracy                           1.00       810
   macro avg       1.00      1.00      1.00       810
weighted avg       1.00      1.00      1.00       810

[[370   2]
 [  2 436]]


In [ ]:
# accuracy
# print("train accuracy",model.score(X_train_std, y_train))
loss, acc = model.evaluate(x_testcnn, Y_test, verbose = 0)
print("accuracy: {:5.2f}%".format(100*acc))


accuracy: 99.51%


In [ ]:
x_testcnn

array([[[-6.5767242e+02],
        [ 5.0445778e+01],
        [-1.2981698e+01],
        ...,
        [ 2.6012712e+00],
        [-2.2783345e-01],
        [-8.1940651e-01]],

       [[-6.8327789e+02],
        [ 5.3257027e+01],
        [-6.4183564e+00],
        ...,
        [ 1.4103930e+00],
        [-1.0183806e+00],
        [-1.8105182e-01]],

       [[-6.6291205e+02],
        [ 3.9057232e+01],
        [-1.9889158e+01],
        ...,
        [-2.2325233e-01],
        [ 5.6215415e+00],
        [ 1.3971729e+01]],

       ...,

       [[-6.2312146e+02],
        [ 6.0755939e+01],
        [-8.2150116e+00],
        ...,
        [-7.5451231e-01],
        [-1.8474494e+00],
        [-1.2289779e+00]],

       [[-7.8434271e+02],
        [ 7.0035851e+01],
        [ 1.3756791e+01],
        ...,
        [-2.7076170e+00],
        [-9.7627831e-01],
        [-9.8998481e-01]],

       [[-4.8801501e+02],
        [ 4.8312027e+01],
        [-2.0366209e+00],
        ...,
        [ 3.0891764e+00],
        [ 3.736

In [ ]:
# saving model to drive
model.save('/content/drive/MyDrive/Colab_Notebooks/gender2.h5')